### Install & Data Setup

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece

In [5]:
!pip install tqdm openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from datasets import load_dataset

dataset_name = "LimYeri/leetcode_with_youtube_captions"
#Importing the dataset
dataset = load_dataset(dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18136 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['cc_content', 'id', 'thumbnail', 'title', 'question_content', 'java', 'c++', 'python', 'javascript', 'title_slug', 'tag', 'level', 'success_rate', 'total_submission', 'total_accepted', 'question_likes', 'question_dislikes', 'question_hints', 'similar_question_ids', 'num_tokens'],
        num_rows: 18136
    })
})

In [8]:
import pandas as pd
df = pd.DataFrame(dataset['train'])

In [9]:
df.head(3)

,cc_content,id,thumbnail,title,question_content,java,c++,python,javascript,title_slug,tag,level,success_rate,total_submission,total_accepted,question_likes,question_dislikes,question_hints,similar_question_ids,num_tokens
0,it has been asked by Facebook Tik Tok it has ...,76,https://i.ytimg.com/vi/XnY8fe0Gs0E/maxresdefau...,Minimum Window Substring,Given two strings `s` and `t` of lengths `m` a...,\n ```java\nimport java.util.HashMap;\n\npu...,\n ```cpp\n#include <string>\n#include <uno...,\n ```python\nfrom collections import Count...,"\n ```javascript\nfunction minWindow(s, t) ...",minimum-window-substring,"Hash Table,String,Sliding Window",Hard,38.9,1890669.0,735482.0,9964.0,529.0,Use two pointers to create a window of letters...,"30,209,239,567,632,727",25434
1,all right let's talk about replace all all ri...,1844,https://i.ytimg.com/vi/cyIhOdKAoUE/sddefault.jpg,Replace All Digits with Characters,You are given a **0-indexed** string `s` that ...,\n ```java\npublic int countBalls(int lowLi...,"\n ```cpp\nint countBalls(int lowLimit, int...",\n ```python\ndef count_balls(low_limit: in...,\n ```javascript\nfunction countBalls(lowLi...,maximum-number-of-balls-in-a-box,"Hash Table,Math,Counting",Easy,73.7,51413.0,37887.0,362.0,85.0,Note that both lowLimit and highLimit are of s...,None,1594
2,hi guys let's talk about a google hi guys let...,1470,https://i.ytimg.com/vi/7HZmP8r1FBE/maxresdefau...,Shuffle the Array,Given the array `nums` consisting of `2n` elem...,\n ```java\nimport java.util.List;\nimport ...,\n ```cpp\n#include <unordered_map>\n#inclu...,\n ```python\nfrom collections import defau...,\n ```javascript\nclass TweetCounts {\n ...,tweet-counts-per-frequency,"Hash Table,Binary Search,Design,Sorting,Ordere...",Medium,43.0,61844.0,26570.0,104.0,198.0,None,None,7345


### chatGPT API

In [10]:
from openai import OpenAI

client = OpenAI(
  organization='your-organization',
  api_key = "your-api-key"
)

In [11]:
over_token = [] # gpt-3.5-turbo-0125
under_token = [] # gpt-4ㅡmodel 중 하나

for i in range(len(df)):
  if df['num_tokens'][i] <= 13000:
    under_token.append(i)
  else:
    over_token.append(i)

In [12]:
print(len(over_token))
print(len(under_token))

1291
16845


In [19]:
def get_openai_response(cc_content):
  prompt = f"""
  Please organize the target text below according to the steps.
  Do not include any additional content that is not written in the target text.
  Separate the steps, but don't miss the details.
  You should explain the algorithm for the coding test problem to the user in detail.
  Ignore text that has nothing to do with the coding test (e.g. promoting YouTube channel, subscribing to channels)

  ## The target text
  {cc_content}
  """

  try:
    completions = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',  # Determines the quality, speed, and cost.
        temperature=0.2,            # Level of creativity in the response
        messages=[
            {"role": "system", "content": "You are a helpful teacher who teaches coding tests."},
            {"role": "user", "content": prompt},
        ],          
        max_tokens=3000,             # Maximum tokens in the prompt AND response
        n=1,                        # The number of completions to generate
        stop=None,                  # An optional setting to control response generation
        top_p=0.2
    )
    return completions.choices[0].message.content
  except Exception as e:
    return "error"

In [18]:
print(len(under_token[:5000]))
print(len(under_token[5000:10000]))
print(len(under_token[10000:]))

5000
5000
6845


In [ ]:
from tqdm import tqdm

for i in tqdm(under_token[:5000]):
    df.loc[i, 'Summary'] = get_openai_response(df.loc[i, 'cc_content'])

  1%|          | 34/5000 [04:13<8:15:04,  5.98s/it]

### Save Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.to_csv('your-path/leetcode_summary.csv')

### Find 'error' string in df['Summary'] and use chatGPT API agian

In [ ]:
error_indices = []
for i, summary in enumerate(df['Summary']):
  if summary == "error":
    error_indices.append(i)
print(error_indices)

In [ ]:
for i in tqdm(error_indices):
    df.loc[i, 'Summary'] = get_openai_response(df.loc[i, 'cc_content'])